# **Retrieval Augmented Generation**

_Taalmodellen zijn heel krachtig, maar zijn qua kennis gelimiteerd tot hun trainingdata en kunnen halucineren. De antwoorden zijn niet gestaafd door een bronvermelding. Vragen over onze eigen documenten kan deze ook niet zomaar beantwoorden. Dit is waar Retrieval Augmented Generation (RAG) een oplossing biedt. Het is een techniek waarbij we onze eigen documenten kunnen bevragen met natuurlijke taal._

<img src="../.github/rag.png" alt="RAG" width="600"/>

---

## **Vector database**

Zoals verteld tijdens de theorie, hebben we een **vector database** nodig om onze documenten in op te slaan.


Dit kan op verschillende manieren:

| Opslagmethode     | Gebruik                   |
|-------------------|---------------------------|
| Cloud             | Productie                 |
| Docker container  | Productie (eigen server)  |
| Lokaal bestand    | Development               |
| In-memory         | Development               |

Wij gaan **Milvus** gebruiken, een open-source vector database.<br>
Net zoals Ollama, kan je Milvus zowel via een CLI als via Python aanspreken.<br>
Naast Milvus zijn er tal van andere opties beschikbaar zoals Chroma, Qdrant, Vespa, (Pinecone, Weaviate) ...


### **1. Installeer de Milvus Python SDK**

In [ ]:
%pip install pymilvus[model] --quiet

### **2. Importeer de benodigde libraries**

In [7]:
from pymilvus import MilvusClient

### **3. Maak een Milvus client aan**

Typisch zou dit er ongeveer zo uitzien:

```py
client = MilvusClient(uri="http://localhost:19530", username="admin", password="password")
```

Wij hebben geen online database die ergens draait en toegankelijk is via een URL. <br>
In plaats daarvan gaan we een **lokaal bestand** aanmaken dat we kunnen gebruiken als database.<br>
Als het bestand niet bestaat, dan wordt het automatisch aangemaakt, handig!

In [ ]:
client = MilvusClient("../milvus.db")

Met het client object kunnen we nu d.m.v. Python code interageren met de Milvus database:
- `client.create_user(...)`
- `client.create_collection(...)`
- `client.insert(...)`
- `client.query(...)`
- ...

### **4. Maak een nieuwe lege collectie aan**

In [ ]:
# Aanmaken van nieuwe collectie
client.create_collection(
  collection_name="artie",
  dimension=5
)

# Lijst alle collecties op ter controle
client.list_collections()

---

## **Documenten in database stoppen**

Nu we onze vector database hebben aangemaakt, kunnen we beginnen met het toevoegen van documenten. <br>
We gaan een aantal PDF's inlezen, omzetten naar embeddings en deze toevoegen aan de database.

### **1. Uitlezen van PDF**

<img src="../.github/tekst-extractie.png" alt="Tekst extractie" width="400"/>

In [ ]:
%pip install pypdf --quiet

In [ ]:
from pypdf import PdfReader

def lees_pdf(path):
    # Open de PDF
    pdf = PdfReader(path)
    text = ""
    # Overloop elke pagina
    for page in pdf.pages:
        # Lees de paginatekst uit
        page_text = page.extract_text()
        # Voeg de paginatekst toe aan de totale tekst
        text += page_text + "\n"
    return text

In [ ]:
text = lees_pdf("artie.pdf")

print(text)

### **2. Tekst omzetten naar vectoren (embedding)**

<img src="../.github/tekst-embedding.png" alt="Tekst embedding" width="400"/>

In [ ]:
def verdeel_in_chunks(text):
    # Kap de tekst in stukken van 512 karakters
    return [text[i:i+512] for i in range(0, len(text), 512)]

chunks = verdeel_in_chunks(text)

print(f"De tekst van {len(text)} karakters is opgedeeld in {len(chunks)} stukken van 512 karakters.")

In [20]:
from pymilvus import model

# Laadt een standaard embedder model
embedder = model.DefaultEmbeddingFunction()

In [ ]:
# Omzetten van teksten naar vectoren
vectors = embedder.encode_documents(chunks)

# Een kijkje nemen naar de eerste vector
print(vectors[0])

### **3. Vectoren in database stoppen**

<img src="../.github/vectors-in-database.png" alt="Tekst embedding" width="500"/>

In [ ]:
from pymilvus import MilvusClient

client = MilvusClient("../milvus.db")

# Formatteer de vectoren als een lijst van dictionaries
data = [ {"text": text, "vector": vector} for text, vector in zip(chunks, vectors) ]

# Vectoren toevoegen aan de artie collectie
client.insert("artie", data)

---

## **Vector database bevragen**

### **1. Vraag embedden**

<img src="../.github/vraag-embedden.png" alt="RAG" width="400"/>

In [ ]:
question = "Wat kan je me vertellen over Art-IE?"

question_vector = embedder.encode_queries([question])[0]

### **2. Relevante documenten zoeken**

<img src="../.github/relevante-docs-zoeken.png" alt="RAG" width="400"/>

In [ ]:
from pymilvus import MilvusClient

client = MilvusClient("../milvus.db")

results = client.search(
    collection_name="artie",
    data=question_vector,
    limit=5,
    output_fields=["text"]
)

print("Resultaten:")
for result in results:
    print(result.text)

### **3. Antwoord formuleren**

In [ ]:
from ollama import chat

def vraag_ollama_rag(context, question, model="gemma3:12b-it-qat"):
    prompt = f"""
Je bent een professionele assistent. Gebruik onderstaande context om de vraag te beantwoorden.
Als het antwoord niet in de context staat, zeg dan dat je het niet weet.

### Context:
{context}

### Vraag:
{question}

### Antwoord:
"""
    response = chat(model=model, prompt=prompt)
    return response.text.strip()

# Chunks van gevonden documenten terug aan elkaar plakken om context te vormen
context = "\n\n".join([result.text for result in results])

# Vraag stellen aan Ollama met de context en de vraag
answer = vraag_ollama_rag(context, question)

print("Antwoord:")
print(answer)